# Writing your own workflow - Equation of State

In this notebook, we will construct our own workflow to calculate the **Equation of State** (EOS). We will again start with
the necessary preparations, such as creating the structure, loading the code, defining the calculation paremeters, and
metadata options.

In [ ]:
%load_ext aiida
%aiida

In [ ]:
from aiida import engine, orm
from aiida_quantumespresso.calculations.pw import PwCalculation
from ase.build import bulk

In [ ]:
# Change the label here to 7.1 when running on nanoHUB
pw_code = orm.load_code('qe-7.3-pw@localhost')
si_structure = orm.StructureData(ase=bulk('Si', 'diamond', a=5.43)).store()

kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([2, 2, 2])

pseudo_family = orm.load_group('SSSP/1.3/PBEsol/efficiency')

cutoff_wfc, cutoff_rho = pseudo_family.get_recommended_cutoffs(
    structure=si_structure,
    unit='Ry'
)

parameters = {
    'CONTROL': {
        'calculation': 'scf',
    },
    'SYSTEM': {
        'occupations': 'smearing',
        'smearing': 'cold',
        'degauss': 0.02
    }
}

metadata_options = {
    "resources": {
        "num_machines": 1
    },
    "max_wallclock_seconds": 1800,
}

***
Now, let's consider the different steps that our workflow should compose:
1. Take an input structure and scale it to both compress and expand it
2. Calculate the SCF energies for all scaled structures
3. Plot the obtained results

Ideally, one would also relax the input structure for scaling, but for simplicity we will omit this step here.

Before going to the actual implementation, it is good to reiterate some of AiiDA's main concepts:

- **Provenance**: AiiDA tracks all data and processes in a provenance graph. To achieve this, data must be in the form
  of AiiDA data types, such as `Int`, `Float`, `StructureData`. This is because these classes inherit from the `Node`
  base class, which contains the functionality to store the data in the database and record the provenance.
- **Calcfunctions**: To run operations that create data, the corresponding Python functions must be converted into AiiDA
  `calcfunction`s, which can be achieved with the `@engine.calcfunction` decorator. It's important to note here, that,
  to keep provenance, these functions need to take AiiDA data `Node`s as inputs and return those as outputs.
- **WorkChains**: To define workflows in AiiDA one has to define a `WorkChain`, which contains the logic of the
  workflow. However, as opposed to `calcfunction`s, `WorkChain`s cannot create data. As such, all data creation has to
  be handled by `calcfunction`s instead, while the `WorkChain` can only *return* the existing data.

With this, let's generate the `calcfunction` to scale the structures:

In [ ]:
@engine.calcfunction
def scale_structures(
    structure: orm.StructureData,  # AiiDA StructureData passed rather than ase.Atoms, which we will obtain internally
    factor_list: orm.List,  # AiiDA orm.List passed, the Python list will be obtained internally for the iteration
    ):

    scaled_structure_dict = {}

    for index, scaling_factor in enumerate(factor_list.get_list()):

        # Here we obtain the ase.Atoms object which we can modify
        # Note that the StructureData cannot be directly scaled, as it is immutable to keep provenance
        ase_structure = structure.get_ase()

        # Scale the ase.Atoms object
        new_cell = ase_structure.get_cell() * scaling_factor
        ase_structure.set_cell(new_cell, scale_atoms=True)

        scaled_structure_dict[f'structure_{index}'] = orm.StructureData(ase=ase_structure)

    return orm.Dict(scaled_structure_dict)

For simple Python base data types, such as `int` or `float`, the explicit conversion to AiiDA data types can also be
omitted, as converting them internally under the hood is possible.

Similarly, we need to define a `calcfunction` that returns the *volumes* and *energies* of the structures, and a
`calcfunction` that fits the EOS equation. The latter step could, in principle, also be done outside of the context of
the `WorkChain` with a normal Python function, however, this step would then **not** be recorded in the provenance of
the full workflow. The implementation of both `calcfunction`s is contained in the next cell, but is not that important,
so we will just skim over it:

In [ ]:
@engine.calcfunction
def create_eos_dictionary(**kwargs) -> orm.Dict:
    eos = {
        label: (result['volume'], result['energy'])
        for label, result in kwargs.items()
    }
    return orm.Dict(eos)


@engine.calcfunction
def fit_eos(volumes: orm.Dict, scf_results: orm.Dict) -> orm.Dict:
    """Fit the EOS of the data."""
    from ase.eos import EquationOfState
    from ase.units import kJ

    volumes_list = []
    energies = []
    for key, data in scf_results.get_dict().items():
        energy = data["energy"]
        energies.append(energy)
        volumes_list.append(volumes.get_dict()[key])
    #
    eos = EquationOfState(volumes_list, energies)
    v0, e0, B = eos.fit()
    # convert B to GPa
    B = B / kJ * 1.0e24
    eos = {"energy unit": "eV", "v0": v0, "e0": e0, "B": B}
    return orm.Dict(eos)

Finally, we define our `EquationOfStateWorkChain`. To this end, we inherit from `engine.WorkChain`. The main method of
our `WorkChain` is the `define` `classmethod`. In it, we define the `input`s, `output`s, and `outline` of our workflow:

```python
```

As you can see above, the outline of the workflow is quite simple: We first run the different SCF calculations, using
the previously defined `calcfunction` to scale our input structure, and then return the results. To run the SCF
calculations, we iterate over the scaled structures and prepare the `PwCalculation`s using the `ProcessBuilder`, as we
did before:

```python
```

In [ ]:
class EquationOfStateWorkChain(engine.WorkChain):
    """WorkChain to compute Equation of State using Quantum ESPRESSO."""

    @classmethod
    def define(cls, spec):
        """Specify inputs and outputs."""
        super().define(spec)
        spec.input("code", valid_type=orm.Code)
        spec.input("structure", valid_type=orm.StructureData)
        spec.input("scale_factors", valid_type=orm.List)

        spec.outline(
            cls.run_eos_scfs,
            cls.results,
        )
        spec.output("eos_dict", valid_type=orm.Dict)

    def run_eos_scfs(self):

        calcjob_dict = {}

        for label, rescaled_structure in scale_structures(self.inputs.structure, self.inputs.scale_factors).items():

            builder = PwCalculation.get_builder()
            builder.code = self.inputs.code
            builder.structure = rescaled_structure
            builder.parameters = orm.Dict(parameters)
            # pseudo_family = orm.load_group('SSSP/1.3/PBEsol/efficiency')
            builder.pseudos = pseudo_family.get_pseudos(structure=rescaled_structure)
            kpoints = orm.KpointsData()
            kpoints.set_kpoints_mesh([2, 2, 2])
            builder.kpoints = kpoints
            builder.metadata.options = metadata_options

            calcjob_dict[label] = self.submit(builder)

        self.ctx.labels = list(calcjob_dict.keys())

        return calcjob_dict

    def results(self):

        self.report(self.ctx)

        # label: self.ctx[label].outputs['properties'] for label in self.ctx.labels

        eos_results = {}
        for label in self.ctx.labels:
            energy = self.ctx[label].outputs.output_parameters.get_dict()['energy']
            volume = self.ctx[label].outputs.output_parameters.get_dict()['volume']
            eos_results[label] = orm.Dict({'energy': orm.Float(energy), 'volume': orm.Float(volume)})

        eos_dict = create_eos_dictionary(**eos_results)
        self.out('eos_dict', eos_dict)

In [ ]:
results = engine.run(
    EquationOfStateWorkChain,
    code=pw_code,
    structure=si_structure,
    scale_factors=orm.List([0.9, 0.95, 1.0, 1.05, 1.1]),
)

In [ ]:
# TODO: Replace this with the actual EOS fit

eos_dict = results['eos_dict'].get_dict()

from matplotlib import pyplot as plt

plt.plot(
    [e[0] for e in eos_dict.values()],
    [v[1] for v in eos_dict.values()],
)